<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Agente_Pesquisador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas google-api-python-client google-generativeai


In [2]:
import pandas as pd
import time
from IPython.display import display, Markdown
from tabulate import tabulate
from urllib.parse import quote_plus
import google.generativeai as genai # SDK do Gemini
import json # Para parsear a resposta do Gemini
from google.colab import userdata # Para acessar a API key

# --- Configuração do Agente e API Key ---
GOOGLE_API_KEY = None
gemini_model_instance = None # Renomeado para clareza

try:
    # O nome padrão para secrets é sem o sufixo _, a menos que você tenha nomeado especificamente assim.
    # Vou usar 'GOOGLE_API_KEY' como padrão, conforme a prática comum.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        display(Markdown("<font color='red'>**Erro Crítico: GOOGLE_API_KEY não encontrada nos Secrets do Colab.** "
                         "Por favor, configure-a com o nome 'GOOGLE_API_KEY'.</font>"))
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        gemini_model_instance = genai.GenerativeModel(
            model_name='gemini-1.5-flash-latest', # Modelo eficiente e capaz
            # A opção response_mime_type pode ser usada com modelos mais recentes
            # e configurações de segurança apropriadas.
            # generation_config=genai.types.GenerationConfig(
            #     response_mime_type="application/json"
            # )
        )
        display(Markdown("✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*"))
except Exception as e:
    display(Markdown(f"<font color='red'>**Erro Crítico ao configurar o SDK do Gemini:** {e}. "
                     "Verifique sua API Key, permissões e se o nome do secret está correto ('GOOGLE_API_KEY'). "
                     "O agente não funcionará sem isso.</font>"))
    # O script poderia parar aqui ou continuar com funcionalidades limitadas se houvesse um fallback.
    # Para este agente, o Gemini é essencial.

# --- Funções Utilitárias ---

def carregar_dados_viagem(caminho_arquivo: str) -> pd.DataFrame | None:
    """
    Carrega os dados de viagem do arquivo CSV e valida as colunas necessárias.
    """
    colunas_requeridas = ['cidade', 'pais', 'data_chegada', 'data_partida', 'hospedagem']
    try:
        df = pd.read_csv(caminho_arquivo)

        # Validar se as colunas requeridas existem
        if not all(col in df.columns for col in colunas_requeridas):
            colunas_faltando = [col for col in colunas_requeridas if col not in df.columns]
            display(Markdown(f"<font color='red'>**Erro: O arquivo `{caminho_arquivo}` não contém as colunas requeridas.** "
                             f"Faltam as seguintes colunas: {', '.join(colunas_faltando)}.</font>"))
            return None

        display(Markdown(f"### Arquivo de Viagem (`{caminho_arquivo}`)"))
        display(df[colunas_requeridas]) # Exibe apenas as colunas relevantes
        return df
    except FileNotFoundError:
        display(Markdown(f"<font color='red'>**Erro: Arquivo `{caminho_arquivo}` não encontrado.** Verifique o nome e o local.</font>"))
        return None
    except Exception as e:
        display(Markdown(f"<font color='red'>**Erro ao carregar o arquivo CSV `{caminho_arquivo}`:** {e}</font>"))
        return None

def gerar_links_pesquisa_google(cidade: str, pais: str) -> tuple[str, str, str]:
    """Gera links de pesquisa úteis para Google Search e Google Maps."""
    query_atracoes = f"principais atrações turísticas em {cidade} {pais}"
    link_google_search = f"https://www.google.com/search?q={quote_plus(query_atracoes)}"
    query_maps_atracoes = f"atrações turísticas em {cidade}, {pais}"
    link_google_maps_atracoes = f"https://www.google.com/maps/search/{quote_plus(query_maps_atracoes)}"
    query_maps_cidade = f"{cidade}, {pais}"
    link_google_maps_cidade = f"https://www.google.com/maps/place/{quote_plus(query_maps_cidade)}"
    return link_google_search, link_google_maps_atracoes, link_google_maps_cidade

# --- Definição do Agente Pesquisador de Atrações ---

class AgentePesquisadorAtracoes:
    """
    Um agente responsável por pesquisar atrações turísticas usando o Gemini
    e fornecer links úteis para pesquisa manual.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def _obter_sugestoes_gemini(self, cidade: str, pais: str, top_n: int = 7) -> list[dict]:
        """
        Ferramenta interna do agente para consultar o Gemini sobre atrações.
        """
        display(Markdown(f"🧠 *Agente consultando Gemini sobre atrações em: **{cidade}, {pais}** (Top {top_n})...*"))

        prompt = f"""
        Você é um assistente de planejamento de viagens altamente especializado e eficiente.
        Sua tarefa é identificar as {top_n} principais e mais recomendadas atrações turísticas para a cidade de {cidade}, localizada em {pais}.

        Critérios para sua seleção:
        1.  **Popularidade e Reconhecimento:** Atrações bem conhecidas e frequentemente visitadas.
        2.  **Qualidade das Avaliações:** Lugares geralmente com avaliações positivas (ex: acima de 4.0/5 estrelas).
        3.  **Relevância Cultural/Histórica:** Locais com significado importante.
        4.  **Diversidade de Experiências:** Inclua uma mistura de tipos, como museus, monumentos, parques, mercados, igrejas/catedrais, mirantes, etc., quando apropriado para a cidade.
        5.  **Singularidade:** Atrações que oferecem uma experiência única ou icônica da cidade/região.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando uma atração,
        contendo EXATAMENTE as seguintes chaves:
        - "nome": (string) O nome oficial e completo da atração.
        - "tipo_principal": (string) A categoria principal da atração (ex: "Museu de Arte", "Catedral Gótica", "Parque Urbano", "Mercado Histórico", "Monumento Nacional", "Mirante Panorâmico"). Seja específico.
        - "descricao_enxuta": (string) Uma descrição concisa e informativa (1-2 frases) que justifique sua inclusão, destacando seus principais atrativos.
        - "avaliacao_geral": (string, opcional) Uma estimativa da avaliação geral se for amplamente conhecida (ex: "Excelente", "Muito Popular", "4.5/5 estrelas"). Use "N/A" se não houver uma estimativa clara.
        - "destaque_principal": (string) O motivo principal pelo qual um turista deveria visitar (ex: "Vistas incríveis da cidade", "Coleção de arte renascentista", "Arquitetura impressionante", "Atmosfera vibrante").

        Exemplo de um item na lista JSON:
        {{
          "nome": "Museu do Louvre",
          "tipo_principal": "Museu de Arte e Antiguidades",
          "descricao_enxuta": "Um dos maiores e mais visitados museus do mundo, lar de obras-primas como a Mona Lisa e a Vênus de Milo.",
          "avaliacao_geral": "4.7/5 estrelas",
          "destaque_principal": "Coleção de arte de renome mundial"
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.4, # Um pouco mais factual, menos aleatório
                    max_output_tokens=3072 # Espaço suficiente para a lista JSON
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            # Tentativa robusta de extrair JSON, mesmo que o modelo adicione ```json ... ```
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para {cidade}.</font>"))
                return []

            atracoes = json.loads(json_text)
            # Garantir que é uma lista, mesmo que o Gemini retorne algo diferente
            if not isinstance(atracoes, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para {cidade}.</font>"))
                 return []

            display(Markdown(f"✅ *Gemini retornou {len(atracoes)} sugestões de atrações para {cidade}.*"))
            return atracoes[:top_n]
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return []
        except Exception as e:
            # Captura de erros mais genéricos, como problemas de API (quota, etc.)
            # ou bloqueios de segurança não esperados.
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return []

    def pesquisar_destino(self, cidade: str, pais: str, top_n_sugestoes: int = 7) -> dict:
        """
        Executa a pesquisa de atrações para um destino específico.
        """
        display(Markdown(f"\n### 🌍 Agente Iniciando Pesquisa para: **{cidade}, {pais}**"))

        links = gerar_links_pesquisa_google(cidade, pais)
        sugestoes_gemini = self._obter_sugestoes_gemini(cidade, pais, top_n=top_n_sugestoes)

        display(Markdown(f"--- Pesquisa para **{cidade}, {pais}** finalizada pelo agente. ---"))
        return {
            "cidade": cidade,
            "pais": pais,
            "link_google_search": links[0],
            "link_google_maps_atracoes": links[1],
            "link_google_maps_cidade": links[2],
            "sugestoes_gemini": sugestoes_gemini
        }

# --- Função Principal de Execução ---
def executar_planejador_viagem(caminho_arquivo_csv: str = 'europa.csv'):
    """
    Orquestra o carregamento dos dados da viagem e a pesquisa de atrações para cada destino.
    Recebe o caminho do arquivo CSV como parâmetro.
    """
    if not GOOGLE_API_KEY or not gemini_model_instance:
        display(Markdown("<font color='red'>**Execução Interrompida.** API Key do Google ou modelo Gemini não configurados corretamente na inicialização.</font>"))
        return

    df_viagem = carregar_dados_viagem(caminho_arquivo_csv)

    if df_viagem is None:
        display(Markdown("<font color='red'>**Não foi possível prosseguir sem os dados da viagem.**</font>"))
        return

    try:
        agente_pesquisador = AgentePesquisadorAtracoes(modelo_llm=gemini_model_instance)
    except ValueError as e:
        display(Markdown(f"<font color='red'>**Erro ao criar o agente pesquisador:** {e}</font>"))
        return

    display(Markdown("\n# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️"))

    resultados_completos = []
    for indice, linha_viagem in df_viagem.iterrows():
        cidade = linha_viagem['cidade']
        pais = linha_viagem['pais']

        resultado_destino = agente_pesquisador.pesquisar_destino(cidade, pais, top_n_sugestoes=15)

        # Adicionar dados da viagem original ao resultado
        resultado_destino['data_chegada'] = linha_viagem['data_chegada']
        resultado_destino['data_partida'] = linha_viagem['data_partida']
        resultado_destino['hospedagem'] = linha_viagem['hospedagem']
        resultados_completos.append(resultado_destino)

        # Exibição imediata por destino
        display(Markdown(f"\n## 📍 Destino: **{resultado_destino['cidade']}, {resultado_destino['pais']}**"))
        display(Markdown(f"🗓️ **Período:** {resultado_destino['data_chegada']} a {resultado_destino['data_partida']}"))
        display(Markdown(f"🏨 **Hospedagem:** {resultado_destino['hospedagem']}"))

        display(Markdown("#### 🔍 Links Úteis para Pesquisa Manual Detalhada (abrem em nova aba):"))
        display(Markdown(f"- **[Google Search: Principais atrações]({resultado_destino['link_google_search']})** ")) # Removido {} extra
        display(Markdown(f"- **[Google Maps: Atrações na área]({resultado_destino['link_google_maps_atracoes']})** ")) # Removido {} extra
        display(Markdown(f"- **[Google Maps: Visão geral da cidade]({resultado_destino['link_google_maps_cidade']})** ")) # Removido {} extra

        if resultado_destino['sugestoes_gemini']:
            display(Markdown("#### ✨ Sugestões de Atrações Principais (via Agente Gemini):"))

            tabela_atracoes_data = []
            # Chaves como definidas no prompt do Gemini
            headers = ["Nº", "Atração", "Tipo Principal", "Descrição Enxuta", "Avaliação Geral", "Destaque Principal"]

            for i, atracao in enumerate(resultado_destino['sugestoes_gemini']):
                tabela_atracoes_data.append([
                    i + 1,
                    atracao.get('nome', 'N/A'),
                    atracao.get('tipo_principal', 'N/A'),
                    atracao.get('descricao_enxuta', 'N/A'),
                    atracao.get('avaliacao_geral', 'N/A'),
                    atracao.get('destaque_principal', 'N/A')
                ])

            tabela_formatada = tabulate(tabela_atracoes_data, headers=headers, tablefmt="pipe", stralign="left")
            display(Markdown(tabela_formatada))
        else:
            display(Markdown("<font color='orange'>⚠️ Nenhuma sugestão de atração específica retornada pelo Gemini para este destino. "
                             "Utilize os links de pesquisa manual acima.</font>"))
        display(Markdown("---")) # Separador visual entre destinos

    # Você pode querer fazer algo com 'resultados_completos' aqui, como salvar em um JSON.
    # Exemplo:
    # with open('planejamento_viagem_atracoes.json', 'w', encoding='utf-8') as f:
    #     json.dump(resultados_completos, f, ensure_ascii=False, indent=4)
    # display(Markdown("\n💾 *Relatório completo também salvo em `planejamento_viagem_atracoes.json` (descomente para ativar).*"))

# --- Ponto de Entrada da Execução ---
if __name__ == "__main__":
    # Certifique-se de que as bibliotecas estão instaladas
    # Em um notebook Colab, você executaria !pip install -q pandas google-generativeai tabulate em uma célula separada.
    # Agora você pode chamar a função passando o caminho do seu arquivo CSV, por exemplo:
    # executar_planejador_viagem('meu_arquivo_viagem.csv')
    # Se nenhum caminho for fornecido, ele usará 'europa.csv' por padrão.
    executar_planejador_viagem()

✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*

### Arquivo de Viagem (`europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Jtália,2025-09-13,2025-09-16,Via Francesco Caracciolo n.7
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hilton Vienna Park



# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️


### 🌍 Agente Iniciando Pesquisa para: **Roma, Jtália**

...

- **[Google Maps: Atrações na área](https://www.google.com/maps/search/atra%C3%A7%C3%B5es+tur%C3%ADsticas+em+Viena%2C+A%C3%BAstria)** 

- **[Google Maps: Visão geral da cidade](https://www.google.com/maps/place/Viena%2C+A%C3%BAstria)** 

#### ✨ Sugestões de Atrações Principais (via Agente Gemini):

|   Nº | Atração                   | Tipo Principal                 | Descrição Enxuta                                                                                                                                              | Avaliação Geral   | Destaque Principal                          |
|-----:|:--------------------------|:-------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------|:--------------------------------------------|
|    1 | Palácio de Hofburg        | Complexo de Palácios Imperiais | Residência de inverno dos Habsburgos por séculos, abriga museus, capelas e apartamentos estatais opulentos.                                                   | Excelente         | Arquitetura imperial e história austríaca   |
|    2 | Palácio de Schönbrunn     | Palácio Barroco e Jardins      | Residência de verão dos Habsburgos, com exuberantes jardins, fontes e um labirinto, representando o auge do estilo barroco.                                   | Excelente         | Magnificência barroca e vastos jardins      |
|    3 | Catedral de Santo Estêvão | Catedral Gótica                | Símbolo de Viena, com sua arquitetura gótica impressionante, torres altas e tesouros históricos.                                                              | Excelente         | Arquitetura gótica e importância histórica  |
|    4 | Museu de História da Arte | Museu de Arte e Antiguidades   | Coleção excepcional de pinturas, esculturas e objetos de arte de diferentes épocas e culturas, incluindo obras mestras de artistas renascentistas e barrocos. | 4.5/5 estrelas    | Coleção de arte de renome mundial           |
|    5 | Albertina                 | Museu de Arte Gráfica          | Coleção extensa de gravuras, desenhos e fotografias, incluindo obras de Dürer, Michelangelo e Picasso.                                                        | 4.5/5 estrelas    | Coleção de arte gráfica excepcional         |
|    6 | Belvedere                 | Museu de Arte e Palácio        | Palácio barroco que abriga a maior coleção mundial de obras de Gustav Klimt, incluindo 'O Beijo'.                                                             | Excelente         | Obras-primas de Gustav Klimt                |
|    7 | Prater                    | Parque de Diversões Histórico  | Parque de diversões icônico com a famosa Roda Gigante, oferecendo diversão e vistas panorâmicas da cidade.                                                    | Muito Popular     | Entretenimento e vistas panorâmicas         |
|    8 | Naschmarkt                | Mercado Histórico              | Mercado vibrante e colorido com uma grande variedade de produtos frescos, especiarias e comidas internacionais.                                               | Muito Popular     | Experiência gastronômica e cultural         |
|    9 | Casa da Música            | Centro de Música Moderna       | Centro de concertos de arquitetura moderna e inovadora, com excelente acústica e programação variada.                                                         | 4.0/5 estrelas    | Concertos e arquitetura moderna             |
|   10 | Schlosspark Schönbrunn    | Jardim Histórico               | Os extensos e belos jardins do Palácio de Schönbrunn, com fontes, esculturas e um labirinto.                                                                  | Excelente         | Paisagismo e tranquilidade                  |
|   11 | Roda Gigante do Prater    | Atração de Entretenimento      | Uma das rodas-gigantes mais famosas do mundo, oferecendo vistas panorâmicas de Viena.                                                                         | Muito Popular     | Vistas panorâmicas de Viena                 |
|   12 | Danúbio                   | Rio e Passeios de Barco        | O rio Danúbio atravessa Viena, oferecendo passeios de barco e vistas cênicas da cidade.                                                                       | Muito Popular     | Passeios de barco e vistas cênicas          |
|   13 | MuseumsQuartier           | Complexo de Museus             | Complexo de museus modernos e contemporâneos, com arquitetura impressionante e diversas exposições.                                                           | 4.0/5 estrelas    | Diversidade de museus e arquitetura moderna |
|   14 | Stephansplatz             | Praça Histórica                | A praça principal de Viena, localizada em frente à Catedral de Santo Estêvão, com atmosfera vibrante e edifícios históricos.                                  | Muito Popular     | Atmosfera vibrante e edifícios históricos   |
|   15 | Volksgarten               | Parque Urbano                  | Um dos parques mais bonitos de Viena, com jardins bem cuidados, monumentos e um ambiente tranquilo.                                                           | Excelente         | Tranquilidade e beleza paisagística         |

---

In [3]:
executar_planejador_viagem("/content/europa.csv")

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Jtália,2025-09-13,2025-09-16,Via Francesco Caracciolo n.7
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hilton Vienna Park



# 🗺️ **Relatório de Pesquisa de Atrações do Agente** ✈️


### 🌍 Agente Iniciando Pesquisa para: **Roma, Jtália**

🧠 *Agente consultando Gemini sobre atrações em: **Roma, Jtália** (Top 15)...*

...

- **[Google Maps: Atrações na área](https://www.google.com/maps/search/atra%C3%A7%C3%B5es+tur%C3%ADsticas+em+Viena%2C+A%C3%BAstria)** 

- **[Google Maps: Visão geral da cidade](https://www.google.com/maps/place/Viena%2C+A%C3%BAstria)** 

#### ✨ Sugestões de Atrações Principais (via Agente Gemini):

|   Nº | Atração                          | Tipo Principal                 | Descrição Enxuta                                                                                                                   | Avaliação Geral   | Destaque Principal                                |
|-----:|:---------------------------------|:-------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------|:------------------|:--------------------------------------------------|
|    1 | Palácio de Schönbrunn            | Palácio Histórico e Jardins    | Residência de verão dos Habsburgos, com exuberantes jardins, fontes e museus, oferecendo um vislumbre da vida imperial.            | Excelente         | Magnificência arquitetônica e paisagística        |
|    2 | Hofburg                          | Complexo de Palácios           | Antiga residência de inverno dos Habsburgos, abrigando museus, capelas e apartamentos imperiais, mostrando a história austríaca.   | Muito Popular     | História imperial e arquitetura grandiosa         |
|    3 | Catedral de São Estêvão          | Catedral Gótica                | Símbolo de Viena, com sua arquitetura gótica impressionante, tesouros religiosos e vistas panorâmicas do topo da torre sul.        | Excelente         | Arquitetura gótica e importância histórica        |
|    4 | Museu de História da Arte        | Museu de Arte                  | Coleção impressionante de pinturas, esculturas e objetos de arte, abrangendo desde a antiguidade até o século XVIII.               | 4.5/5 estrelas    | Coleção abrangente de obras-primas                |
|    5 | Belvedere                        | Museu de Arte e Palácio        | Palácio barroco com dois edifícios, abrigando a maior coleção de obras de Gustav Klimt, incluindo 'O Beijo'.                       | Excelente         | Obras-primas de Gustav Klimt                      |
|    6 | Prater                           | Parque de Diversões            | Parque de diversões icônico, lar da famosa roda-gigante Wiener Riesenrad, oferecendo entretenimento e vistas deslumbrantes.        | Muito Popular     | Entretenimento e vistas panorâmicas               |
|    7 | Naschmarkt                       | Mercado Histórico              | Um dos maiores e mais antigos mercados ao ar livre da Europa, oferecendo uma variedade de produtos frescos, especiarias e comidas. | 4.0/5 estrelas    | Experiência gastronômica e cultural vibrante      |
|    8 | Casa da Música                   | Centro de Música e Arquitetura | Centro de concertos moderno com arquitetura inovadora, oferecendo uma variedade de concertos e eventos musicais.                   | 4.5/5 estrelas    | Arquitetura moderna e concertos de alta qualidade |
|    9 | Palácio do Belvedere Inferior    | Palácio Barroco e Museu        | Palácio barroco que abriga uma coleção de arte austríaca, incluindo obras de Gustav Klimt e Egon Schiele.                          | Excelente         | Arte austríaca e arquitetura barroca              |
|   10 | Roda Gigante Wiener Riesenrad    | Atração Turística e Mirante    | Roda gigante icônica no Prater, oferecendo vistas panorâmicas de Viena e uma experiência nostálgica.                               | Muito Popular     | Vistas panorâmicas de Viena                       |
|   11 | Espaço de Arte Leopold           | Museu de Arte Moderna          | Museu dedicado à arte austríaca do século XX, com uma coleção impressionante de obras expressionistas e modernistas.               | 4.5/5 estrelas    | Arte moderna austríaca                            |
|   12 | Jardins do Palácio de Schönbrunn | Jardim Histórico               | Extensos jardins barrocos com fontes, esculturas, e um labirinto, oferecendo um ambiente tranquilo e paisagístico.                 | Excelente         | Paisagismo barroco e ambiente sereno              |
|   13 | Stephansplatz                    | Praça Histórica                | Praça central de Viena, localizada em frente à Catedral de São Estêvão, com atmosfera vibrante e edifícios históricos.             | Muito Popular     | Atmosfera vibrante e localização central          |
|   14 | Museu Albertina                  | Museu de Arte Gráfica          | Coleção extensa de gravuras, desenhos e fotografias, incluindo obras de artistas renomados como Michelangelo e Picasso.            | 4.5/5 estrelas    | Coleção abrangente de arte gráfica                |
|   15 | Danúbio                          | Rio e Paisagem Urbana          | Rio que atravessa Viena, oferecendo passeios de barco, vistas cênicas e oportunidades para relaxamento ao ar livre.                | N/A               | Vistas cênicas e atividades ao ar livre           |

---